In [1]:
pip install chromadb

In [2]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import os
import json
import chromadb
from sentence_transformers import SentenceTransformer
from google.colab import drive  # For Google Colab users

# File paths
chunks_file = "/content/drive/MyDrive/APPLICATION/TfL_Traffic_Data/tfl_cleaned_chunks.json"
chroma_db_path = "/content/drive/MyDrive/APPLICATION/TfL_Traffic_Data/chroma_db"

# Step 2: Load preprocessed text chunks
with open(chunks_file, "r", encoding="utf-8") as file:
    chunks = json.load(file)


In [4]:

# Step 3: Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path=chroma_db_path)
collection = chroma_client.get_or_create_collection(name="tfl_data")

# Step 5: Embed and store each chunk in ChromaDB
for i, chunk in enumerate(chunks):
    embedding = model.encode(chunk).tolist()
    collection.add(
        ids=[str(i)],  # Unique ID for each chunk
        embeddings=[embedding],
        documents=[chunk]
    )

print(f"✅ {len(chunks)} chunks successfully embedded and stored in ChromaDB!")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 23 chunks successfully embedded and stored in ChromaDB!


In [12]:
# Step 6: Test retrieval with a sample query
query = "Tell me about TfL?"
query_embedding = model.encode(query).tolist()

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

print("\n🔍 Top 3 Retrieved Results:")
for doc in results["documents"][0]:
    print(f"- {doc[:200]}...")  # Show first 200 characters of each result


🔍 Top 3 Retrieved Results:
- over the last 25 years 25 years of TfL 25 key improvements weve made to our network over the last 25 years Photocard applications All photocard applications are now open and you can request refunds wh...
- works and events in London which may affect your journey Major works and events Report a crime or incident Sustainability Check the best times to travel TfL Go app Experience London blog Made by TfL b...
- heathrow-express good service hull-trains good service london-north-eastern-railway good service lumo good service merseyrail good service northern-rail good service scotrail good service south-wester...
